In [1]:
!pip install --quiet transformers streamlit pyngrok ##quiet tells pip to reduce its console output so only minimal messages are shown, this basically makes the collab notebook more readable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.5 MB/s eta 0:00:00


For advanced AI models we're downloading transformers, to which we will give a simpoe UI using Streamlit. To make a tunnel between colab and Streamlit app we use pyngrok.

In [2]:
%%writefile summarizer_app.py
import streamlit as st
from transformers import pipeline

# Page Title
st.title('📰 AI Text Summarizer (3-Sentence)')

# Info
st.write('Paste any news article or blog post below and click **Summarize**.')

# Text input
input_text = st.text_area('Input text:', height=300)

# Summarization logic (runs when button pressed)
if st.button('Summarize'):
    if not input_text.strip():
        st.warning('Please paste some text to summarize!')
    else:
        # Load the model only once per session (caches for speed)
        @st.cache_resource
        def load_summarizer():
            return pipeline('summarization', model='facebook/bart-large-cnn')
        summarizer = load_summarizer()
        # Call the model
        summary = summarizer(
            input_text,
            max_length=130,   # total tokens for 3-4 sentences
            min_length=30,
            do_sample=False
        )[0]['summary_text']
        st.subheader('Summary:')
        st.success(summary)


Writing summarizer_app.py


In [3]:
from pyngrok import ngrok
import threading
import time
import os

# Optional: read token from env for safety; fallback to placeholder
authtoken = os.getenv("NGROK_AUTHTOKEN", "33d4o3ecCJQGucW1W7ItQiLRuMZ_3G53hrvWtMxBcxJtmeEUV")  # replace if not using env

# Set the token once (persists to ngrok config for the session)
ngrok.set_auth_token(authtoken)

# Kill old tunnels (if any)
ngrok.kill()

# Start Streamlit in a new thread
def run():
    os.system('streamlit run summarizer_app.py --server.port 8501 --server.headless true')

thread = threading.Thread(target=run, daemon=True)
thread.start()

# Wait for Streamlit to start
time.sleep(5)

# Open HTTP tunnel to port 8501 (Streamlit default)
tunnel = ngrok.connect(8501, "http")
print('Access your Streamlit app here:')
print(tunnel.public_url)


Access your Streamlit app here:
https://stratiformis-quartus-tammy.ngrok-free.dev


In [4]:
from google.colab import files
files.download('summarizer_app.py')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>